In [12]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import pandas as pd
import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata

session = cc.database.create_session()

In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:41835 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
input_path = '/g/data/e14/erd561/NOAA_OI_SST_V2/'
output_path = '/g/data/e14/erd561/Australia_All/'

data = xr.open_dataset(input_path + 'sst.mnmean.nc')

data

<xarray.Dataset>
Dimensions:    (lat: 180, lon: 360, nbnds: 2, time: 447)
Coordinates:
  * lat        (lat) float32 89.5 88.5 87.5 86.5 ... -86.5 -87.5 -88.5 -89.5
  * lon        (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * time       (time) datetime64[ns] 1981-12-01 1982-01-01 ... 2019-02-01
Dimensions without coordinates: nbnds
Data variables:
    sst        (time, lat, lon) float32 ...
    time_bnds  (time, nbnds) datetime64[ns] ...
Attributes:
    title:          NOAA Optimum Interpolation (OI) SST V2
    Conventions:    CF-1.0
    history:        Wed Apr  6 13:47:45 2005: ncks -d time,0,278 SAVEs/sst.mn...
    comments:       Data described in  Reynolds, R.W., N.A. Rayner, T.M.\nSmi...
    platform:       Model
    source:         NCEP Climate Modeling Branch
    institution:    National Centers for Environmental Prediction
    References:     https://www.esrl.noaa.gov/psd/data/gridded/data.noaa.oiss...
    dataset_title:  NOAA Optimum Interpolation (OI) SST V

In [23]:
lonW = 88
lonE = 182

latS = -72
latN = 22

temp_z0 = xr.open_dataset(input_path + 'sst.mnmean.nc').\
                sst.\
                sel(lon=slice(lonW,lonE)).\
                sel(lat=slice(latS,latN,-1)).\
                groupby('time.year').\
                mean('time', skipna=True).sel(year=slice(1982,2018))

temp_z0 = temp_z0.assign_coords(lon=temp_z0.lon-360)

mask = xr.open_dataset(input_path + 'lsmask.nc').\
                mask.\
                sel(lon=slice(lonW,lonE)).\
                sel(lat=slice(latS,latN,-1))

mask = mask.assign_coords(lon=mask.lon-360)

temp_z0 = temp_z0.where(mask==1, np.nan)

temp_z0 = temp_z0.squeeze()
print(temp_z0)

<xarray.DataArray 'sst' (year: 37, lat: 94, lon: 94)>
array([[[       nan,        nan,        nan, ..., -1.4008332,
         -1.3941666, -1.3916665],
        [       nan,        nan,        nan, ..., -1.5083333,
         -1.485    , -1.4791666],
        [       nan,        nan,        nan, ..., -1.5458332,
         -1.5141665, -1.5024999],
        ...,
        [28.170832 , 28.106665 , 27.987501 , ..., 26.448334 ,
         26.415833 , 26.339167 ],
        [27.9925   , 27.940002 , 27.843332 , ..., 26.20833  ,
         26.162498 , 26.067497 ],
        [27.866667 , 27.814165 , 27.698332 , ..., 25.955    ,
         25.918335 , 25.820833 ]],

       [[       nan,        nan,        nan, ..., -1.3008333,
         -1.3275   , -1.3416667],
        [       nan,        nan,        nan, ..., -1.3008333,
         -1.3316666, -1.3475   ],
        [       nan,        nan,        nan, ..., -1.2175   ,
         -1.2524999, -1.2683333],
        ...,
        [28.204168 , 28.115831 , 28.039167 , ..., 26.3

In [24]:
lat = temp_z0.lat
lon = temp_z0.lon
year = temp_z0.year

# temp_z0_trans = np.transpose(temp_z0, (2,1,0))
temp_z0_trans = temp_z0
print(np.shape(temp_z0_trans))
temp_z0_slope = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='temp_z0')
temp_z0_p_value = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='temp_z0')
temp_z0_std_err = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='temp_z0')
for iid, i in enumerate(lat):
    for jid, j in enumerate(lon):
        temp_z0_slope[jid, iid], _, _, temp_z0_p_value[jid, iid], temp_z0_std_err[jid, iid] = \
        stats.linregress(year, temp_z0_trans[:, jid, iid])
    print('lat ' + str(np.array(i)))
print(temp_z0_slope)

(37, 94, 94)
lat -71.5
lat -70.5
lat -69.5
lat -68.5
lat -67.5
lat -66.5
lat -65.5
lat -64.5
lat -63.5
lat -62.5
lat -61.5
lat -60.5
lat -59.5
lat -58.5
lat -57.5
lat -56.5
lat -55.5
lat -54.5
lat -53.5
lat -52.5
lat -51.5
lat -50.5
lat -49.5
lat -48.5
lat -47.5
lat -46.5
lat -45.5
lat -44.5
lat -43.5
lat -42.5
lat -41.5
lat -40.5
lat -39.5
lat -38.5
lat -37.5
lat -36.5
lat -35.5
lat -34.5
lat -33.5
lat -32.5
lat -31.5
lat -30.5
lat -29.5
lat -28.5
lat -27.5
lat -26.5
lat -25.5
lat -24.5
lat -23.5
lat -22.5
lat -21.5
lat -20.5
lat -19.5
lat -18.5
lat -17.5
lat -16.5
lat -15.5
lat -14.5
lat -13.5
lat -12.5
lat -11.5
lat -10.5
lat -9.5
lat -8.5
lat -7.5
lat -6.5
lat -5.5
lat -4.5
lat -3.5
lat -2.5
lat -1.5
lat -0.5
lat 0.5
lat 1.5
lat 2.5
lat 3.5
lat 4.5
lat 5.5
lat 6.5
lat 7.5
lat 8.5
lat 9.5
lat 10.5
lat 11.5
lat 12.5
lat 13.5
lat 14.5
lat 15.5
lat 16.5
lat 17.5
lat 18.5
lat 19.5
lat 20.5
lat 21.5
<xarray.DataArray 'temp_z0' (lat: 94, lon: 94)>
array([[        nan,         nan,        

In [27]:
temp_z0_xr = xr.DataArray(np.transpose(temp_z0.values, (1,2,0)), name='temp_z0_oisst',
                        coords=[lat, lon, year], 
                        dims=['lat', 'lon', 'year'])
print(temp_z0_xr)
temp_z0_xr.to_netcdf(output_path + 'temp_z0_oisst.nc')

temp_z0_slope_xr = xr.DataArray(temp_z0_slope, name='temp_z0_slope_oisst',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(temp_z0_slope_xr)
temp_z0_slope_xr.to_netcdf(output_path + 'temp_z0_slope_oisst.nc')

temp_z0_p_value_xr = xr.DataArray(temp_z0_p_value, name='temp_z0_p_value_oisst',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(temp_z0_p_value_xr)
temp_z0_p_value_xr.to_netcdf(output_path + 'temp_z0_p_value_oisst.nc')

temp_z0_std_err_xr = xr.DataArray(temp_z0_std_err, name='temp_z0_std_err_oisst',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(temp_z0_std_err_xr)
temp_z0_std_err_xr.to_netcdf(output_path + 'temp_z0_std_err_oisst.nc')

<xarray.DataArray 'temp_z0_oisst' (lat: 94, lon: 94, year: 37)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [-1.4008332, -1.3008333, -1.3008333, ..., -1.4291667,
         -1.3766667, -1.4291667],
        [-1.3941666, -1.3275   , -1.3349999, ..., -1.4408334,
         -1.4025   , -1.4158331],
        [-1.3916665, -1.3416667, -1.3516666, ..., -1.4583334,
         -1.4016666, -1.4191666]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [-1.5083333, -1.3008333, -1.36     ,

In [26]:
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_z0_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=0, method='nearest')-273.15
print(temp_z0_1_mod_month)

temp_z0_1_mod_month.load()
print(temp_z0_1_mod_month)

xt_ocean = temp_z0_1_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_z0_1_mod_month = temp_z0_1_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(temp_z0_1_mod_month)

temp_z0_1_mod = temp_z0_1_mod_month.groupby('time.year').mean('time')
print(temp_z0_1_mod)

# temp_z0_1_mod = temp_z0_1_mod.sel(year=slice(2004+60*4,2017+60*4))
# print(temp_z0_1_mod)

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp' (time: 300, yt_ocean: 160, xt_ocean: 90)>
dask.array<sub, shape=(300, 160, 90), dtype=float32, chunksize=(1, 124, 90), chunktype=numpy.ndarray>
Coordinates:
    st_ocean  float64 1.152
  * xt_ocean  (xt_ocean) float64 -269.5 -268.5 -267.5 ... -182.5 -181.5 -180.5
  * yt_ocean  (yt_ocean) float64 -69.63 -69.18 -68.71 ... 17.85 18.59 19.36
  * time      (time) object 1958-07-02 12:00:00 ... 2257-07-02 12:00:00
<xarray.DataArray 'temp' (time: 300, yt_ocean: 160, xt_ocean: 90)>
array([[[       nan,        nan,        nan, ..., -1.3452759,
         -1.3441772, -1.342804 ],
        [       nan,        nan,        nan, ..., -1.3128967,
         -1.3044739, -1.3048401],
        [       nan,        nan,        nan, ..., -1.2649231,
         -1.258606 , -1.2553406],
        ...,
        [27.779205 , 27.703247 , 27.695343 , ..., 26.297943 ,
         26.258667 , 26.213867 ],
        [27.70932  , 27.692932 , 27.660675 , ..., 26.196869 ,
         26.152313 , 26.096619 ],
   

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [15]:
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_z0_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=0, method='nearest')-273.15
print(temp_z0_025_mod_month)

temp_z0_025_mod_month.load()
print(temp_z0_025_mod_month)

xt_ocean = temp_z0_025_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_z0_025_mod_month = temp_z0_025_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(temp_z0_025_mod_month)

temp_z0_025_mod = temp_z0_025_mod_month.groupby('time.year').mean('time')
print(temp_z0_025_mod)

025deg_jra55v13_iaf_gmredi6


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp' (time: 300, yt_ocean: 475, xt_ocean: 360)>
dask.array<sub, shape=(300, 475, 360), dtype=float32, chunksize=(1, 216, 248), chunktype=numpy.ndarray>
Coordinates:
    st_ocean  float64 1.152
  * xt_ocean  (xt_ocean) float64 -269.9 -269.6 -269.4 ... -180.6 -180.4 -180.1
  * yt_ocean  (yt_ocean) float64 -69.99 -69.88 -69.78 ... 19.49 19.72 19.96
  * time      (time) object 1958-06-30 12:00:00 ... 2257-06-30 12:00:00
<xarray.DataArray 'temp' (time: 300, yt_ocean: 475, xt_ocean: 360)>
array([[[       nan,        nan,        nan, ..., -1.3691101,
         -1.3626709, -1.355896 ],
        [       nan,        nan,        nan, ..., -1.3651428,
         -1.3577881, -1.3512878],
        [       nan,        nan,        nan, ..., -1.3556519,
         -1.3484192, -1.3435669],
        ...,
        [27.802582 , 27.755707 , 27.710144 , ..., 25.934357 ,
         25.9169   , 25.897644 ],
        [27.709442 , 27.649994 , 27.596466 , ..., 25.882812 ,
         25.862396 , 25.845978 ],

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'temp' (year: 300, yt_ocean: 475, xt_ocean: 360)>
array([[[       nan,        nan,        nan, ..., -1.3691101,
         -1.3626709, -1.355896 ],
        [       nan,        nan,        nan, ..., -1.3651428,
         -1.3577881, -1.3512878],
        [       nan,        nan,        nan, ..., -1.3556519,
         -1.3484192, -1.3435669],
        ...,
        [27.802582 , 27.755707 , 27.710144 , ..., 25.934357 ,
         25.9169   , 25.897644 ],
        [27.709442 , 27.649994 , 27.596466 , ..., 25.882812 ,
         25.862396 , 25.845978 ],
        [27.600006 , 27.545258 , 27.514069 , ..., 25.834686 ,
         25.809082 , 25.799255 ]],

       [[       nan,        nan,        nan, ..., -1.6878967,
         -1.6752625, -1.658081 ],
        [       nan,        nan,        nan, ..., -1.6947937,
         -1.6841736, -1.6699829],
        [       nan,        nan,        nan, ..., -1.6965942,
         -1.6852722, -1.6719666],
        ...,
        [27.432068 , 27.406036 , 27.3960

In [16]:
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_z0_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=0, method='nearest')-273.15
print(temp_z0_01_mod_month)

temp_z0_01_mod_month.load()
print(temp_z0_01_mod_month)

xt_ocean = temp_z0_01_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_z0_01_mod_month = temp_z0_01_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(temp_z0_01_mod_month)

temp_z0_01_mod = temp_z0_01_mod_month.groupby('time.year').mean('time')
print(temp_z0_01_mod)

01deg_jra55v13_iaf


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp' (time: 396, yt_ocean: 1186, xt_ocean: 900)>
dask.array<sub, shape=(396, 1186, 900), dtype=float32, chunksize=(1, 300, 400), chunktype=numpy.ndarray>
Coordinates:
    st_ocean  float64 0.5413
  * xt_ocean  (xt_ocean) float64 -269.9 -269.8 -269.7 ... -180.2 -180.1 -180.0
  * yt_ocean  (yt_ocean) float64 -70.0 -69.96 -69.92 ... 19.75 19.84 19.94
  * time      (time) object 1985-01-14 12:00:00 ... 2017-12-14 12:00:00
<xarray.DataArray 'temp' (time: 396, yt_ocean: 1186, xt_ocean: 900)>
array([[[       nan,        nan,        nan, ..., -1.6495056,
         -1.6503601, -1.6513062],
        [       nan,        nan,        nan, ..., -1.647522 ,
         -1.6482849, -1.6491699],
        [       nan,        nan,        nan, ..., -1.6450806,
         -1.6460876, -1.6468506],
        ...,
        [25.293152 , 25.223358 , 25.163635 , ..., 25.029602 ,
         25.062805 , 25.099335 ],
        [25.10736  , 25.045837 , 24.992981 , ..., 25.03833  ,
         25.077118 , 25.118774

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'temp' (year: 33, yt_ocean: 1186, xt_ocean: 900)>
array([[[       nan,        nan,        nan, ..., -1.7491251,
         -1.7482351, -1.7479095],
        [       nan,        nan,        nan, ..., -1.7536646,
         -1.7519506, -1.7504578],
        [       nan,        nan,        nan, ..., -1.757871 ,
         -1.7560654, -1.7536036],
        ...,
        [27.634636 , 27.625414 , 27.618845 , ..., 26.017271 ,
         26.008379 , 26.005022 ],
        [27.61979  , 27.614256 , 27.60399  , ..., 26.008963 ,
         26.004587 , 25.99981  ],
        [27.59926  , 27.58808  , 27.559431 , ..., 26.000448 ,
         26.000229 , 25.996185 ]],

       [[       nan,        nan,        nan, ..., -1.6110916,
         -1.6088562, -1.6058298],
        [       nan,        nan,        nan, ..., -1.6142935,
         -1.6126913, -1.610583 ],
        [       nan,        nan,        nan, ..., -1.616633 ,
         -1.6149164, -1.613645 ],
        ...,
        [27.499376 , 27.499647 , 27.4988

In [21]:
temp_z0_1_mod_xr = xr.DataArray(temp_z0_1_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='temp_z0_1_mod',
                        coords=[temp_z0_1_mod.yt_ocean, temp_z0_1_mod.xt_ocean, temp_z0_1_mod.year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
temp_z0_1_mod_xr.to_netcdf(output_path + 'temp_z0_1_mod.nc')


temp_z0_025_mod_xr = xr.DataArray(temp_z0_025_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='temp_z0_025_mod',
                        coords=[temp_z0_025_mod.yt_ocean, temp_z0_025_mod.xt_ocean, temp_z0_025_mod.year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
temp_z0_025_mod_xr.to_netcdf(output_path + 'temp_z0_025_mod.nc')


temp_z0_01_mod_xr = xr.DataArray(temp_z0_01_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='temp_z0_01_mod',
                        coords=[temp_z0_01_mod.yt_ocean, temp_z0_01_mod.xt_ocean, temp_z0_01_mod.year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
temp_z0_01_mod_xr.to_netcdf(output_path + 'temp_z0_01_mod.nc')